In [117]:
import os
with open('bigram_chunks_complete_formulation_with_pauses_2.txt') as f:
    t = f.read()

In [121]:
#une fonction qui crée une liste de chunks
def chunker(text):
    #une chaine représentant un chunk, liste de chunks
    nCh = ""
    listDeChunkers = []
    for i in range(len(text)):
        #des conditions pour fermer le chunk, pour diviser les chunks d'une chaine de car / text reperé consecutivement
        if text[i-1] == ")" and text[i] == " " or text[i-1] == ")" and text[i] == "\n" or text[i] == " " and text[i+1] == "(" or text[i] == "\n" and text[i+1] == "(":
            listDeChunkers.append(nCh)
            nCh = ""
        #condition nécessaire pour fermer le chunk precedent et ajouter un @ qui se trouve entre les chunks
        elif text[i-1] == ")" and text[i] == "@" and text[i+1] == "(": 
            if nCh!=")":
                listDeChunkers.append(nCh)
            listDeChunkers.append("@")
            nCh=""
        #condition nécessaire pour un @ qui se trouve entre les deux balises ouvrantes --> résultat de SEM
        elif text[i] == "@" and text[i-1] == "(" and text[i+1] == "(":
            listDeChunkers.append("@")
            nCh=""
        else: 
            nCh+=text[i]
    return listDeChunkers

chunks = chunker(t)
print(chunks)

['', "(NP L'/DET arrêt/NC )", '(PP du/P+D tabac/NC )', '(VN est/)', '@', '(NP un/DET sujet/NC )', '(NP qui/PROREL )', '(VN est/V )', '(AdP  de_plus_en_plus/ADV )', '(AdP fréquent/ADV )', '', '(NP un/DET sujet/NC )', '(NP qui/PROREL )', '(VN est/V )', '(AdP  de_plus_en_plus/ADV )', '(AdP fréquent/)', '@', '(PP dans/P les/DET conversations/NC )', '(PP en/P France/NPP )', ',/PONCT', '(PP où/PROREL )', '(NP encore/ADV )', '', '(PP dans/P les/DET conversations/NC )', '(PP en/P France/NPP )', ',/PONCT', '(PP où/PROREL )', '(AdP encore/)', '@', '(AdP équent/ADV )', '', '(NP équent/@/NC de_plus_en_plus/ADV )', '(NP répandu/VPP )', '', '(AdP  de_plus_en_plus/ADV )', '(VN répandu/)', '@', '(us_en_plus/ADV )', '(VN répandu/VPP )', '(PP en/P France/NPP )', './PONCT', '(NP L/NC )', '', '(PP en/P France/NPP )', './PONCT', '(NP L/@/DET es/NC différentes/ADJ )', '', '(NP es/DET différentes/@/ADJ e/NC )', './PONCT', "(NP L'/DET Etat/NPP )", '', '(NP e/NC )', './PONCT', "(NP L'/DET Etat/)", '@', '( Etat

In [122]:
#fonction qui nous donne accès à l'étiquette du chunk, à la nature du chunk
#il faut lui donner en entre un chunk duquel nous voulons savoir l'étiquette
def prefix(ch):
    nch = ""
    for l in ch:
        if l == " ":
            break
            return nch
        if l == "(":
            pass
        else: 
            nch+=l
    return nch

#l'enlevement des PONCT qui empechent l'analyse et des element vides comme chunk vide () 
chunksNettoyes=[]
for ele in chunks : 
    if ele != "" and ele != "()" and "PONCT" not in ele : 
        chunksNettoyes.append(ele)
print(chunksNettoyes)


["(NP L'/DET arrêt/NC )", '(PP du/P+D tabac/NC )', '(VN est/)', '@', '(NP un/DET sujet/NC )', '(NP qui/PROREL )', '(VN est/V )', '(AdP  de_plus_en_plus/ADV )', '(AdP fréquent/ADV )', '(NP un/DET sujet/NC )', '(NP qui/PROREL )', '(VN est/V )', '(AdP  de_plus_en_plus/ADV )', '(AdP fréquent/)', '@', '(PP dans/P les/DET conversations/NC )', '(PP en/P France/NPP )', '(PP où/PROREL )', '(NP encore/ADV )', '(PP dans/P les/DET conversations/NC )', '(PP en/P France/NPP )', '(PP où/PROREL )', '(AdP encore/)', '@', '(AdP équent/ADV )', '(NP équent/@/NC de_plus_en_plus/ADV )', '(NP répandu/VPP )', '(AdP  de_plus_en_plus/ADV )', '(VN répandu/)', '@', '(us_en_plus/ADV )', '(VN répandu/VPP )', '(PP en/P France/NPP )', '(NP L/NC )', '(PP en/P France/NPP )', '(NP L/@/DET es/NC différentes/ADJ )', '(NP es/DET différentes/@/ADJ e/NC )', "(NP L'/DET Etat/NPP )", '(NP e/NC )', "(NP L'/DET Etat/)", '@', '( Etat/NPP )', '(VN a/V )', '(VN a/)', '@', '(AdP même/ADV )', '(VN crée/VPP )', '(AdP même/ADV )', '(VN

In [125]:
#ce bout de code parcourt les chunks et une fois il rencontre @ tout seule
#il repère l'étiquette de chunk avant et après sous la condition que l'étiquette
#est confrome au standard d'étiquetage, sinon c'est compté comme les données bruitées. 
#les étiquettes possibles sont pris de siteweb de SEM.

étiquettes = ["NP", "PP", "AdP", "AP", "CONJ", "VN", "__UNKNOWN__"]
bruuit=0
listeEntreChunks = []
#parcours d'une liste de chunks
for i in range (len(chunksNettoyes)-1) :
    #if élément d'une liste est un arobase et 
    if chunksNettoyes[i] == "@" :
        #extractions d'étiquettes
        prefix1 = prefix(chunksNettoyes[i-1])
        prefix2 = prefix(chunksNettoyes[i+1])
        #si les deux étiquettes sont conforme au standard on les compte --> les données sûres
        if prefix1 in étiquettes and prefix2 in étiquettes :
            key = (prefix1,prefix2)
            listeEntreChunks.append(key)
            print('bon :', chunksNettoyes[i-1], chunksNettoyes[i], chunksNettoyes[i+1] )
        else:
            print('bruuuuit :', chunksNettoyes[i-1], chunksNettoyes[i], chunksNettoyes[i+1] )
            bruuit+=1

bon : (VN est/) @ (NP un/DET sujet/NC )
bon : (AdP fréquent/) @ (PP dans/P les/DET conversations/NC )
bon : (AdP encore/) @ (AdP équent/ADV )
bruuuuit : (VN répandu/) @ (us_en_plus/ADV )
bruuuuit : (NP L'/DET Etat/) @ ( Etat/NPP )
bon : (VN a/) @ (AdP même/ADV )
bon : (VN crée/) @ (NP différentes/DET mesures/NC )
bruuuuit : (PP dans/P les/DET lieux/NC publics/) @ (garette/NC )
bruuuuit : (NP la/DET cigarette/NC électronique/) @ (V )
bon : (AdP encore/) @ (NP la/DET création/NC )
bruuuuit : (PP de/P paquet/NC neutre/ADJ ) @ (NCT
@
 )
bruuuuit : (NCT
@
 ) @ (T )
bruuuuit : (PP de/P ces/) @ (ET mesures/NC )
bon : (NP s/) @ (VN permettent/V )
bon : (PP de/P diminuer/) @ (AdP vraiment/ADV )
bruuuuit : (VN ne/ADV voulant/VPR pas/) @ (DV mettre/VINF )
bruuuuit : (PP dans/P les/DET cigarettes/) @ (NC pré-faites/ADJ )
bruuuuit : (PP à/P roulées/NC ) @ (NCT
 @
bon : (CONJ Mais/) @ (NP la/DET création/NC )
bon : (CONJ ou/) @ (CONJ ou/CC )
bruuuuit : (VN ne/ADV marchent/V pas/) @ (C )
bon : (NP un

In [124]:
#ce bout de code genère le dico avec des résultats d'analyse pour le cas où les pauses arrivent entre les chunks      
dicoEntre = {}
for ele in listeEntreChunks : 
    n=0
    for i in range(len(listeEntreChunks)):
        if ele == listeEntreChunks[i] : 
            n+=1
    dicoEntre[ele] = n
dicoEntre['données bruitées'] = bruuit
print(dicoEntre)

#pour la formulation : 
#1237 données de dico concaténés. Données bruités c-à-d l'étiquette donnée par SEM n'est pas conforme au standard de chunking
#ou chunker a fait n'importe quoi, mais cela ne veut pas dire que les pauses ne sont pas entre les chunks
#dans la fênetre avant on peut voir des cas des bruits

{('VN', 'NP'): 89, ('AdP', 'PP'): 23, ('AdP', 'AdP'): 19, ('VN', 'AdP'): 30, ('AdP', 'NP'): 47, ('NP', 'VN'): 114, ('PP', 'AdP'): 24, ('CONJ', 'NP'): 44, ('CONJ', 'CONJ'): 2, ('NP', 'PP'): 95, ('PP', 'PP'): 92, ('PP', 'VN'): 66, ('NP', 'NP'): 130, ('NP', 'AdP'): 22, ('AP', 'PP'): 16, ('PP', 'CONJ'): 44, ('AdP', 'VN'): 13, ('PP', 'NP'): 136, ('NP', 'AP'): 3, ('AdP', 'AP'): 6, ('VN', 'VN'): 26, ('VN', 'PP'): 55, ('CONJ', 'AdP'): 8, ('CONJ', 'PP'): 6, ('NP', 'CONJ'): 35, ('AdP', 'CONJ'): 10, ('VN', 'AP'): 18, ('PP', 'AP'): 1, ('AP', 'NP'): 16, ('VN', 'CONJ'): 19, ('AP', 'CONJ'): 13, ('CONJ', 'VN'): 9, ('AP', 'AP'): 1, ('AP', 'VN'): 4, ('AP', 'AdP'): 3, 'données bruitées': 811}


In [127]:
#nettoyage de données bruites (les lettres séparés, chunks vides) et des espaces entre les chunks
#nettoyage des cas ou la pause arrive avec la ponctuation ou entre les chunks et est étiquetté comme le ponct
chunksComplex = []
for ele in chunks : 
    if len(ele) > 7 : 
        if "PONCT" not in ele : 
            chunksComplex.append(ele)
    
#ce bout de code repère l'info dedans quel chunks il y a des pauses
#aussi le nombre des données difficiles à analyser
#il retourne la liste de chunks qui contient la pause dedans
étiquettes = ["NP", "PP", "AdP", "AP", "CONJ", "VN", "__UNKNOWN__"]
listeInsideChunk = []
bruit = 0
for ele in chunksComplex : 
    if "@" in ele : 
        nature = prefix(ele)
        if nature in étiquettes : 
            print(nature, 'in chunk :', ele)
            listeInsideChunk.append(nature)
        else : 
            print("données bruites :", ele)
            bruit+=1

NP in chunk : (NP équent/@/NC de_plus_en_plus/ADV )
NP in chunk : (NP L/@/DET es/NC différentes/ADJ )
NP in chunk : (NP es/DET différentes/@/ADJ e/NC )
données bruites : (NCT
@
 )
VN in chunk : (VN augmentent/@/V s/VPP )
PP in chunk : (PP sur/P les/DET paquet/@/NC s/ADJ )
NP in chunk : (NP s/@/ET )
NP in chunk : (NP égions/@/DET régions/NC proches/ADJ )
PP in chunk : (PP de/P la/DET frontière/@/NC s/ADJ )
NP in chunk : (NP les/@/DET t/NC )
PP in chunk : (PP dans/@/P la/DET consommation/NC )
PP in chunk : (PP pour/P diminuer/@/VINF )
PP in chunk : (PP en/@/P d/NC )
AdP in chunk : (AdP  de_plus_@_en_plus/ADV )
PP in chunk : (PP de/P douze/DET ans/@/NC )
PP in chunk : (PP du/P+D prix/@/NC d/ADJ )
NP in chunk : (NP satisfaisant/@/ADJ donnent/NC )
NP in chunk : (NP les/@/DET Ces/NC )
NP in chunk : (NP Ces/@/DET moyens/NC )
NP in chunk : (NP ils/@/CLS )
PP in chunk : (PP de/P consommateurs/@/NC )
PP in chunk : (PP chez/P les/DET jeunes/@/NC )
NP in chunk : (NP un/DET salaire/@/NC )
NP in chu

In [128]:
dicoInside = {}
for ele in listeInsideChunk : 
    n=0
    for i in range(len(listeInsideChunk)):
        if ele == listeInsideChunk[i] : 
            n+=1
    dicoInside[ele] = n
dicoInside['donné bruités'] = bruit
print(dicoInside)

#NP in chunk (NP les/@/DET t/NC )
#PP in chunk (PP de/P cigarette/NC neutre/@/ADJ s/NC )

{'NP': 369, 'VN': 64, 'PP': 329, 'AdP': 57, 'AP': 25, 'CONJ': 17, 'donné bruités': 7}
